### BG-NBD ve Gamma-Gamma ile CLTV Tahmini

In [2]:
import pandas as pd
import datetime as dt
# !pip install lifetimes
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
# !pip install scikit-learn
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.mode.chained_assignment = None

### Görev 1: Veriyi Hazırlama

In [3]:
# Adım 1: flo_data_20K.csv verisini okuyunuz.

flo2 = pd.read_csv('flo_data_20k.csv')
flo2.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00,1.00,139.99,799.38,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.00,2.00,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.00,2.00,189.97,395.35,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.00,1.00,39.99,81.98,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.00,1.00,49.99,159.99,[AKTIFSPOR]


In [4]:
# Adım 2: Aykırı değerleri baskılamak için gerekli olan outlier_thresholds ve replace_with_thresholds fonksiyonlarını tanımlayınız.
# Not: cltv hesaplanırken frequency değerleri integer olması gerekmektedir.Bu nedenle alt ve üst limitlerini round() ile yuvarlayınız.
# fonksiyonlar
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = round(low_limit,0)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = round(up_limit,0)

In [5]:
# Adım 3: "order_num_total_ever_online", "order_num_total_ever_offline", "customer_value_total_ever_offline",
# "customer_value_total_ever_online" değişkenlerinin aykırı değerleri varsa baskılayanız.

columns = ["order_num_total_ever_online",
           "order_num_total_ever_offline",
           "customer_value_total_ever_offline",
           "customer_value_total_ever_online"]
for column in columns:
    replace_with_thresholds(flo2, column)

In [6]:
# Adım 4: Omnichannel müşterilerin hem online'dan hem de offline platformlardan alışveriş yaptığını ifade etmektedir. Her bir müşterinin toplam
# alışveriş sayısı ve harcaması için yeni değişkenler oluşturunuz.

flo2['total_order_num'] = flo2['order_num_total_ever_online'] + flo2['order_num_total_ever_offline']
flo2['total_customer_value'] = flo2['customer_value_total_ever_online'] + flo2['customer_value_total_ever_offline']
flo2.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_order_num,total_customer_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00,1.00,139.99,799.38,[KADIN],5.00,939.37
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.00,2.00,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.00,2013.55
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.00,2.00,189.97,395.35,"[ERKEK, KADIN]",5.00,585.32
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.00,1.00,39.99,81.98,"[AKTIFCOCUK, COCUK]",2.00,121.97
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.00,1.00,49.99,159.99,[AKTIFSPOR],2.00,209.98


In [8]:
# Adım 5: Değişken tiplerini inceleyiniz. Tarih ifade eden değişkenlerin tipini date'e çeviriniz.
flo2.dtypes

,0
master_id,object
order_channel,object
last_order_channel,object
first_order_date,object
last_order_date,object
last_order_date_online,object
last_order_date_offline,object
order_num_total_ever_online,float64
order_num_total_ever_offline,float64
customer_value_total_ever_offline,float64


In [9]:
date_columns = ['first_order_date', 'last_order_date', 'last_order_date_online', 'last_order_date_offline']
for column in date_columns:
    flo2[column] = pd.to_datetime(flo2[column])
flo2.dtypes

,0
master_id,object
order_channel,object
last_order_channel,object
first_order_date,datetime64[ns]
last_order_date,datetime64[ns]
last_order_date_online,datetime64[ns]
last_order_date_offline,datetime64[ns]
order_num_total_ever_online,float64
order_num_total_ever_offline,float64
customer_value_total_ever_offline,float64


### Görev 2: CLTV Veri Yapısının Oluşturulması

In [13]:
# Adım 1: Veri setindeki en son alışverişin yapıldığı tarihten 2 gün sonrasını analiz tarihi olarak alınız.

flo2["last_order_date"].max()
analysis_date = dt.datetime(2021,6,1)

In [17]:
# Adım 2: customer_id, recency_cltv_weekly, T_weekly, frequency ve monetary_cltv_avg değerlerinin yer aldığı yeni bir cltv dataframe'i
# oluşturunuz. Monetary değeri satın alma başına ortalama değer olarak, recency ve tenure değerleri ise haftalık cinsten ifade edilecek.

cltv_df = pd.DataFrame()
cltv_df["customer_id"] = flo2["master_id"]
cltv_df["recency_cltv_weekly"] = ((flo2["last_order_date"]- flo2["first_order_date"]).dt.days) / 7
cltv_df["recency_cltv_weekly"].head()

,recency_cltv_weekly
0,17.00
1,209.86
2,52.29
3,1.57
4,83.14


In [18]:
cltv_df["T_weekly"] = ((analysis_date - flo2["first_order_date"]).dt.days) / 7
cltv_df["T_weekly"].head()

,T_weekly
0,30.57
1,224.86
2,78.86
3,20.86
4,95.43


In [22]:
cltv_df["frequency"] = flo2["total_order_num"]
cltv_df["monetary_cltv_avg"] = flo2["total_customer_value"] / flo2["total_order_num"]
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.57,5.00,187.87
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,224.86,21.00,95.88
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,78.86,5.00,117.06
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,20.86,2.00,60.98
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.43,2.00,104.99


In [23]:
# Adım 1: BG/NBD modelini fit ediniz.
#    • 3 ay içerisinde müşterilerden beklenen satın almaları tahmin ediniz ve exp_sales_3_month olarak cltv
# dataframe'ine ekleyiniz.

bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df['frequency'], cltv_df['recency_cltv_weekly'], cltv_df['T_weekly'])
cltv_df['exp_sales_3_month'] = bgf.predict(12, cltv_df['frequency'], cltv_df['recency_cltv_weekly'], cltv_df['T_weekly'])
cltv_df[['customer_id', 'exp_sales_3_month']].head()

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


,customer_id,exp_sales_3_month
0,cc294636-19f0-11eb-8d74-000d3a38a36f,0.97
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,0.98
2,69b69676-1a40-11ea-941b-000d3a38a36f,0.67
3,1854e56c-491f-11eb-806e-000d3a38a36f,0.70
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,0.40


In [24]:
#    • 6 ay içerisinde müşterilerden beklenen satın almaları tahmin ediniz ve exp_sales_6_month olarak cltv
# dataframe'ine ekleyiniz.

cltv_df['exp_sales_6_month'] = bgf.predict(26, cltv_df['frequency'], cltv_df['recency_cltv_weekly'], cltv_df['T_weekly'])
cltv_df[['customer_id', 'exp_sales_6_month']].head()

,customer_id,exp_sales_6_month
0,cc294636-19f0-11eb-8d74-000d3a38a36f,2.11
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,2.13
2,69b69676-1a40-11ea-941b-000d3a38a36f,1.45
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.52
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,0.86


In [25]:
# Adım 2: Gamma-Gamma modelini fit ediniz. Müşterilerin ortalama bırakacakları değeri tahminleyip exp_average_value olarak cltv
# dataframe'ine ekleyiniz.

ggf = GammaGammaFitter(penalizer_coef=0.001)
ggf.fit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])
cltv_df['exp_average_value'] = ggf.conditional_expected_average_profit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])
cltv_df[['customer_id', 'exp_average_value']].head()

,customer_id,exp_average_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,188.47
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,96.26
2,69b69676-1a40-11ea-941b-000d3a38a36f,118.40
3,1854e56c-491f-11eb-806e-000d3a38a36f,65.72
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,108.59


In [26]:
# Adım 3: 6 aylık CLTV hesaplayınız ve cltv ismiyle dataframe'e ekleyiniz.
#    • Cltv değeri en yüksek 20 kişiyi gözlemleyiniz.

cltv_df['cltv'] = cltv_df['exp_sales_6_month'] * cltv_df['exp_average_value']
top_20_cltv = cltv_df.nlargest(20, 'cltv')
top_20_cltv[['customer_id', 'cltv']]

,customer_id,cltv
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,3288.06
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,3225.44
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,1721.33
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,1699.23
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,1677.73
8868,9ce6e520-89b0-11ea-a6e7-000d3a38a36f,1642.03
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,1565.36
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,1527.11
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,1456.56
4157,7eed6468-4540-11ea-acaf-000d3a38a36f,1389.78


### Görev 4: CLTV Değerine Göre Segmentlerin Oluşturulması

In [27]:
# Adım 1: 6 aylık CLTV'ye göre tüm müşterilerinizi 4 gruba (segmente) ayırınız ve grup isimlerini veri setine ekleyiniz.
cltv_df['cltv_segment'] = pd.cut(cltv_df['cltv'],
                                   bins=[-float("inf"), 100, 500, 1000, float("inf")],
                                   labels=['Düşük', 'Orta', 'Yüksek', 'Çok Yüksek'])
cltv_df[['customer_id', 'cltv', 'cltv_segment']].head()

,customer_id,cltv,cltv_segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,397.70,Orta
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,205.05,Orta
2,69b69676-1a40-11ea-941b-000d3a38a36f,172.03,Orta
3,1854e56c-491f-11eb-806e-000d3a38a36f,99.74,Düşük
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,93.18,Düşük


# Adım 2: 4 grup içerisinden seçeceğiniz 2 grup için yönetime kısa kısa 6 aylık aksiyon önerilerinde bulununuz.

- düşük olanlar için: özel indirim ya da promosyonlar sunarak motivasyonlarını arttırmaya çalışabiliriz.

- çok yüksek olanlar için : puan biriktirme sistemi gibi isteklerinin yukarda tutmaya sağlayacak avantajlar ek imkanlar sağlayabiliriz yapabiliriz.